In [1]:
# Useful for defining quantities
from astropy import units as u

# Earth focused modules, ISS example orbit and time span generator
from poliastro.earth import EarthSatellite
from poliastro.earth.plotting import GroundtrackPlotter
from poliastro.twobody import Orbit
from poliastro.spacecraft import Spacecraft
from poliastro.examples import iss
from poliastro.util import time_range
from poliastro.bodies import Earth

In [ ]:
import numpy as np

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider
import ipywidgets as widgets

In [25]:
radius_E = 6400

In [51]:
km2deg(100,100,45)

(0.015784264402068565, 0.029845481523317945)

In [47]:
def km2deg(dlatkm, dlonkm, lat):
    dlat = np.deg2rad(dlatkm/110.574)
    dlon = np.deg2rad(dlonkm/111.320/np.cos(lat))
    
    return dlat, dlon

In [57]:
@interact(h=FloatSlider(min=7000, max=19000, step=10, continuous_update=False),
          i=FloatSlider(min=0, max=90, step=1, continuous_update=False),
          duration=IntSlider(min=10, max=1000, step=10, continuous_update=False),
          max_day_bn_rep=IntSlider(value=2,min=1, max=30, step=1, continuous_update=False),
          ground_size=FloatSlider(min=0, max=90,value=20.,continuous_update=False))
def groundtract(h, i, duration,max_day_bn_rep, ground_size):
    # Build spacecraft instance
    a = (radius_E+h) * u.km
    ecc = 0 * u.one
    inc = i * u.deg
    raan = 0 * u.deg
    argp = 0 * u.deg
    nu = 0 * u.deg
    orb = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)
    print("Original period: ", orb.period.to(u.h))
    #adjust to integer orbits in a day
    T_E =  Earth.rotational_period.to(u.s)
    ratio =T_E/orb.period 
    ratio = np.floor(max_day_bn_rep*ratio)/float(max_day_bn_rep)*u.one
    new_period = T_E/ratio
    a = (Earth.k*(new_period/2/np.pi)**2)**(1./3)
    orb = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

    print("Adjusted period: ", orb.period.to(u.h), "Adjusted altitude: ", (a - radius_E*u.km).to(u.km))
    sat = EarthSatellite(orb, None)
    t_span = time_range(orb.epoch, periods=duration*5, end=orb.epoch+duration * u.h)
    gp = GroundtrackPlotter()
    gp.update_layout(title="Groundtrack")

    # Plot previously defined EarthSatellite object
    gp.plot(
        sat,
        t_span,
        label="Sat",
        color = "red",
    )
    data = gp.fig.data[1]
    theta = np.linspace(2*np.pi,0)
    
    for lat, lon in zip(data['lat'],data['lon']):
        gp.fig.add_scattergeo(marker=None,fill='toself',fillcolor='blue', 
                      lat=lat+ground_size*np.sin(theta),
                      lon=lon+ground_size*np.cos(theta),opacity=0.3)
    gp.fig.show()

interactive(children=(FloatSlider(value=7000.0, continuous_update=False, description='h', max=19000.0, min=700…

In [4]:
gp.update_geos(projection_type="orthographic")
gp.fig.show()

NameError: name 'gp' is not defined

In [34]:
a = 7000 * u.km
ecc = 0 * u.one
inc = 10 * u.deg
raan = 0 * u.deg
argp = 0 * u.deg
nu = 0 * u.deg
orb = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)
print("Original period: ", orb.period.to(u.h))
sat = EarthSatellite(orb, None)
t_span = time_range(orb.epoch, periods=50, end=orb.epoch+10 * u.h)
gp = GroundtrackPlotter()
gp.update_layout(title="Groundtrack")

# Plot previously defined EarthSatellite object
gp.plot(
    sat,
    t_span,
    label="Sat",
    color = "red",
    marker=None,
)
lat, lon = data

gp.fig.add_scattergeo(fill='toself',fillcolor='blue', 
                      lat=[np.flipud(30+20*np.sin(theta)),np.flipud(-30+20*np.sin(theta))],
                      lon=[np.flipud(20+20*np.cos(theta)),np.flipud(20+20*np.cos(theta))],opacity=0.3)

Original period:  1.6190323993572264 h


In [30]:
gp.fig.data[1]['lat']

array([-1.44275407e-03,  7.09757804e+00,  9.99757386e+00,  6.91112344e+00,
       -2.61436781e-01, -7.28124105e+00, -9.99057777e+00, -6.71786117e+00,
        5.24135947e-01,  7.45986024e+00,  9.97653016e+00,  6.51996418e+00,
       -7.86477120e-01, -7.63330629e+00, -9.95544121e+00, -6.31757313e+00,
        1.04828296e+00,  7.80145338e+00,  9.92732657e+00,  6.11083146e+00,
       -1.30937648e+00, -7.96417927e+00, -9.89220734e+00, -5.89988526e+00,
        1.56958114e+00,  8.12136541e+00,  9.85011005e+00,  5.68488322e+00,
       -1.82872088e+00, -8.27289708e+00, -9.80106664e+00, -5.46597648e+00,
        2.08662021e+00,  8.41866346e+00,  9.74511445e+00,  5.24331860e+00,
       -2.34310428e+00, -8.55855773e+00, -9.68229611e+00, -5.01706539e+00,
        2.59799895e+00,  8.69247715e+00,  9.61265954e+00,  4.78737488e+00,
       -2.85113086e+00, -8.82032318e+00, -9.53625788e+00, -4.55440718e+00,
        3.10232750e+00,  8.94200154e+00])

In [35]:
theta = np.linspace(0,2*np.pi)
#theta = np.hstack(theta, theta[0])
gp.fig.add_scattergeo(fill='toself',fillcolor='blue', 
                      lat=[np.flipud(30+20*np.sin(theta)),np.flipud(-30+20*np.sin(theta))],
                      lon=[np.flipud(20+20*np.cos(theta)),np.flipud(20+20*np.cos(theta))],opacity=0.3)

In [18]:
import numpy as np
theta = np.linspace(0,2*np.pi)
theta = np.hstack([theta, theta[0]])
theta.np.f

array([0.        , 0.12822827, 0.25645654, 0.38468481, 0.51291309,
       0.64114136, 0.76936963, 0.8975979 , 1.02582617, 1.15405444,
       1.28228272, 1.41051099, 1.53873926, 1.66696753, 1.7951958 ,
       1.92342407, 2.05165235, 2.17988062, 2.30810889, 2.43633716,
       2.56456543, 2.6927937 , 2.82102197, 2.94925025, 3.07747852,
       3.20570679, 3.33393506, 3.46216333, 3.5903916 , 3.71861988,
       3.84684815, 3.97507642, 4.10330469, 4.23153296, 4.35976123,
       4.48798951, 4.61621778, 4.74444605, 4.87267432, 5.00090259,
       5.12913086, 5.25735913, 5.38558741, 5.51381568, 5.64204395,
       5.77027222, 5.89850049, 6.02672876, 6.15495704, 6.28318531,
       0.        ])

In [37]:
theta

array([6.28318531, 6.15495704, 6.02672876, 5.89850049, 5.77027222,
       5.64204395, 5.51381568, 5.38558741, 5.25735913, 5.12913086,
       5.00090259, 4.87267432, 4.74444605, 4.61621778, 4.48798951,
       4.35976123, 4.23153296, 4.10330469, 3.97507642, 3.84684815,
       3.71861988, 3.5903916 , 3.46216333, 3.33393506, 3.20570679,
       3.07747852, 2.94925025, 2.82102197, 2.6927937 , 2.56456543,
       2.43633716, 2.30810889, 2.17988062, 2.05165235, 1.92342407,
       1.7951958 , 1.66696753, 1.53873926, 1.41051099, 1.28228272,
       1.15405444, 1.02582617, 0.8975979 , 0.76936963, 0.64114136,
       0.51291309, 0.38468481, 0.25645654, 0.12822827, 0.        ])